Modified from https://lectures.quantecon.org/jl/optgrowth.html

In [1]:
using QuantEcon, Distributions, NamedTuples, Optim

## Expectations of Random Variables
* See https://github.com/jlperla/ECON407_2018/blob/master/notebooks/code_examples.ipynb for examples of higher-order functions, anonymous functions, etc.
* See https://github.com/jlperla/ECON407_2018/blob/master/notebooks/expectations_integration.ipynb for more details on numeric integration and expectation operators

In [2]:
#This sort of general utility could be added to the Distributions library or an extension library
#While it is useful to understand how it works, library users should generally not need to write it

#A Normal specialization of the `expectation` function
function expectation(dist::Distributions.Normal{T}; n=20, kws...) where {T}
    nodes, weights = qnwnorm(n, [mean(dist)],[var(dist)])
    return f -> dot(f.(nodes), weights)  #Returns a function (of a function f)
end

#Convenience to call with the function directly.
function expectation(f, dist::D; kws...) where {D <: UnivariateDistribution}
    E = expectation(dist; kws...) #Gets the appropriate expectation operator for dist
    return E(f) #calls the expectation with f
end

#Example calculating the variance numerically.
E = expectation(Normal(0.2, 1.5))
testvar = E(x -> x^2) - (E(x -> x))^2
@show testvar ≈ var(Normal(0.2, 1.5)) #Test versus the built-in variance of a normal

testvar ≈ var(Normal(0.2, 1.5)) = true


true

## Setting up Parameters
Set up a grid and a named tuple of parameters for calculations.

In [3]:
#Setup a set of named parameters
α = 0.4
β = 0.96
μ = 0
s = 0.1
f(k) = k^α #Production function
u(c) = log(c) #Utilility Function
dist = Normal(μ, s^2)
E = expectation(dist) #convenience operator for expectations

# Grid, used for different Calculations.
grid_max = 4         # Largest grid point
grid_size = 200      # Number of grid points
grid_y = collect(linspace(1e-5, grid_max, grid_size))
c_min = 1E-10 #The minimum allowed c value

params = @NT(grid = grid_y, β = β, u = u, f = f, E = E, dist = dist, c_min = c_min) #In julia v0.7, the @NT is not necessary


(grid = [1.0e-5, 0.0201105, 0.0402109, 0.0603114, 0.0804118, 0.100512, 0.120613, 0.140713, 0.160814, 0.180914  …  3.8191, 3.8392, 3.8593, 3.8794, 3.8995, 3.9196, 3.9397, 3.9598, 3.9799, 4.0], β = 0.96, u = u, f = f, E = #2, dist = Distributions.Normal{Float64}(μ=0.0, σ=0.010000000000000002), c_min = 1.0e-10)

## Setting up Parameters and Solving the Closed Form
See (12) and https://lectures.quantecon.org/jl/optgrowth.html#an-example

In [4]:
#Note: taking the parameters directly because this function does not support general functions, etc.
function analyticalsolution(α, β, μ)
    c_star(y) = (1 - α * β) * y
    v_star(y) = log(1 - α * β) / (1 - β) + (μ + α * log(α * β)) / (1 - α) * (1 / (1 - β) - 1 / (1 - α * β)) + 1 / (1 - α * β) * log(y)
    return v_star, c_star 
end
v_star, c_star = analyticalsolution(α, β, μ)

(v_star, c_star)

## Define the Bellman Operator
Sere (11) and https://lectures.quantecon.org/jl/optgrowth.html#the-bellman-operator

This uses linear interpolation of the expectation operator to be defined

In [7]:
#The Bellman Operator (11)
function T(w_grid, params)
    grid, β, u, f, E, c_min = params.grid, params.β, params.u, params.f, params.E, params.c_min #unpack
    
    w = LinInterp(grid, w_grid) #linear interpolation of w_grid

    return [-optimize(c -> - ( #Negates because this is a minimizer
                u(c) + β * E( ζ -> w(f(y - c) * exp(ζ)) ) #objective
                ), c_min, y).minimum for y in grid] #for grid
end

#A More verbose version of this funciton
function T_verbose(w_grid, params)
    grid, β, u, f, E, c_min = params.grid, params.β, params.u, params.f, params.E, params.c_min #unpack
    
    w = LinInterp(grid, w_grid) #linear interpolation of w_grid
    wprime_grid = similar(w_grid) #Preallocating the result to be similar to w_grid
    
    for i in 1:length(w_grid)
        y = grid[i] #At each point on the grid
        c_max = y #i.e. the maximum value of c is y

        yprime(c, ζ) = f(y - c) * exp(ζ) #Law of motion for y
        objective(c) =  - (u(c) + β * E(ζ -> w(yprime(c, ζ)))) #objective negated to become a minimum
        
        result = optimize(objective, c_min, c_max);
        wprime_grid[i] = -result.minimum #negated to become the maximum again
    end
    return wprime_grid
end

T_verbose (generic function with 1 method)

## Checking The Operator on the Analytic Solution

In [8]:
#Plotting the results
#w = T(v_star.(grid_y), params)
w = T_verbose(v_star.(grid_y), params)
using Plots
gr()
plot(grid_y, w, lw=2, alpha=0.6, label="Tv^*")
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="v^*")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
<polyline clip-path="url(#clip9602)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,128.92 41.6256,113.602 44.3601,104.614 47.0946,98.2584 49.829,93.321 52.5635,89.2858 55.298,85.8744 58.0324,82.918 60.7669,80.3092 
 63.5014,77.9785 66.2358,75.8723 68.9703,73.945 71.7047,72.1739 74.4392,70.5358 77.1737,69.0074 79.9081,67.5812 82.6426,66.2381 85.3771,64.975 88.1115,63.7773 
 90.846,62.6438 93.5805,61.5632 96.3149,60.535 99.0494,59.5505 101.784,58.6096 104.518,57.7062 107.253,56.8382 109.987,56.0038 112.722,55.1985 115.456,54.4226 
 118.191,53.6724 120.925,52.9466 123.66,52.2447 126.394,51.5637 129.128,50.9029 131.863,50.2621 134.597,49.6386 137.332,49.0322 140.066,48.4426 142.801,47.8677 
 145.535,47.3073 148.27,46.7614 151.004,46.2282 153.739,45.7074 156.473,45.1989 159.208,44.7018 161.942,44.2153 164.677,43.7395 167.411,43.2739 170.145,42.8177 
 172.88,42.3706 175.614,41.9326 178.349,41.5031 181.083,41.0815 183.818,40.6679 186.552,40.2621 189.287,39.8634 192.021,39.4717 194.756,39.087 197.49,38.7088 
 200.225,38.337 202.959,37.9713 205.693,37.6115 208.428,37.2576 211.162,36.9092 213.897,36.5662 216.631,36.2283 219.366,35.8957 222.1,35.5679 224.835,35.2449 
 227.569,34.9265 230.304,34.6127 233.038,34.3033 235.773,33.9982 238.507,33.6972 241.241,33.4002 243.976,33.1071 246.71,32.8179 249.445,32.5325 252.179,32.2506 
 254.914,31.9723 257.648,31.6975 260.383,31.426 263.117,31.1579 265.852,30.8928 268.586,30.6311 271.321,30.3723 274.055,30.1165 276.79,29.8637 279.524,29.6137 
 282.258,29.3665 284.993,29.122 287.727,28.8803 290.462,28.6411 293.196,28.4045 295.931,28.1704 298.665,27.9386 301.4,27.7095 304.134,27.4825 306.869,27.2579 
 309.603,27.0355 312.338,26.8154 315.072,26.5974 317.806,26.3816 320.541,26.1678 323.275,25.9561 326.01,25.7464 328.744,25.5387 331.479,25.3329 334.213,25.129 
 336.948,24.927 339.682,24.7267 342.417,24.5283 345.151,24.3316 347.886,24.1367 350.62,23.9435 353.354,23.752 356.089,23.562 358.823,23.3738 361.558,23.187 
 364.292,23.0019 367.027,22.8183 369.761,22.6362 372.496,22.4556 375.23,22.2765 377.965,22.0988 380.699,21.9224 383.434,21.7477 386.168,21.574 388.903,21.402 
 391.637,21.231 394.371,21.0616 397.106,20.8933 399.84,20.7264 402.575,20.5606 405.309,20.3961 408.044,20.2329 410.778,20.0707 413.513,19.9099 416.247,19.7501 
 418.982,19.5916 421.716,19.4341 424.451,19.2778 427.185,19.1225 429.919,18.9683 432.654,18.8152 435.388,18.6631 438.123,18.5122 440.857,18.3621 443.592,18.2132 
 446.326,18.0652 449.061,17.9182 451.795,17.7722 454.53,17.627 457.264,17.483 459.999,17.3397 462.733,17.1975 465.467,17.0561 468.202,16.9156 470.936,16.7761 
 473.671,16.6373 476.405,16.4995 479.14,16.3625 481.874,16.2264 484.609,16.0911 487.343,15.9565 490.078,15.8229 492.812,15.69 495.547,15.5579 498.281,15.4267 
 501.016,15.2961 503.75,15.1664 506.484,15.0374 509.219,14.9091 511.953,14.7816 514.688,14.6548 517.422,14.5287 520.157,14.4034 522.891,14.2788 525.626,14.1548 
 528.36,14.0316 531.095,13.909 533.829,13.7871 536.564,13.6659 539.298,13.5453 542.032,13.4254 544.767,13.3062 547.501,13.1875 550.236,13.0695 552.97,12.9522 
 555.705,12.8354 558.439,12.7193 561.174,12.6037 563.908,12.4888 566.643,12.3745 569.377,12.2607 572.112,12.1476 574.846,12.035 577.58,11.923 580.315,11.8116 
 
 "/>
<polyline clip-path="url(#clip9602)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,297.182 38.8912,128.905 41.6256,113.576 44.3601,104.607 47.0946,98.2437 49.829,93.3076 52.5635,89.2744 55.298,85.8644 58.0324,82.9105 60.7669,80.3049 
 63.5014,77.9741 66.2358,75.8656 68.9703,73.9408 71.7047,72.17 74.4392,70.5306 77.1737,69.0043 79.9081,67.5766 82.6426,66.2354 85.3771,64.9709 88.1115,63.7748 
 90.846,62.6401 93.5805,61.5607 96.314

## Showing the Evolution from an Initial Condition

In [9]:
# An example initial condition with a plot
initial_w = 5 * log.(grid_y)

w = copy(initial_w) 
p = plot(grid_y, w, label = "w")

n = 21
plotevery = 5
for i = 1:n
    w = T(w, params)
    if(i % plotevery == 0)  #modulus
        plot!(grid_y, w, label = "T^$i w")    
    end
end
   
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.8, label="Analytic")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
<polyline clip-path="url(#clip9802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,375.869 38.8912,161.191 41.6256,141.635 44.3601,130.194 47.0946,122.076 49.829,115.779 52.5635,110.634 55.298,106.283 58.0324,102.515 60.7669,99.1909 
 63.5014,96.2175 66.2358,93.5276 68.9703,91.072 71.7047,88.813 74.4392,86.7215 77.1737,84.7744 79.9081,82.953 82.6426,81.242 85.3771,79.6288 88.1115,78.1029 
 90.846,76.6553 93.5805,75.2783 96.3149,73.9654 99.0494,72.7109 101.784,71.5097 104.518,70.3576 107.253,69.2507 109.987,68.1856 112.722,67.1592 115.456,66.1688 
 118.191,65.212 120.925,64.2866 123.66,63.3906 126.394,62.5221 129.128,61.6796 131.863,60.8614 134.597,60.0664 137.332,59.2931 140.066,58.5405 142.801,57.8074 
 145.535,57.0928 148.27,56.3959 151.004,55.7158 153.739,55.0517 156.473,54.4029 159.208,53.7686 161.942,53.1483 164.677,52.5414 167.411,51.9472 170.145,51.3652 
 172.88,50.795 175.614,50.2362 178.349,49.6881 181.083,49.1505 183.818,48.623 186.552,48.1051 189.287,47.5966 192.021,47.097 194.756,46.6062 197.49,46.1237 
 200.225,45.6494 202.959,45.1829 205.693,44.724 208.428,44.2724 211.162,43.8279 213.897,43.3903 216.631,42.9595 219.366,42.535 222.1,42.1169 224.835,41.7049 
 227.569,41.2988 230.304,40.8985 233.038,40.5037 235.773,40.1144 238.507,39.7304 241.241,39.3516 243.976,38.9778 246.71,38.6089 249.445,38.2447 252.179,37.8851 
 254.914,37.5301 257.648,37.1795 260.383,36.8332 263.117,36.4911 265.852,36.1531 268.586,35.8191 271.321,35.489 274.055,35.1627 276.79,34.8402 279.524,34.5213 
 282.258,34.2059 284.993,33.8941 287.727,33.5856 290.462,33.2805 293.196,32.9787 295.931,32.68 298.665,32.3845 301.4,32.092 304.134,31.8025 306.869,31.516 
 309.603,31.2323 312.338,30.9515 315.072,30.6734 317.806,30.3981 320.541,30.1254 323.275,29.8553 326.01,29.5878 328.744,29.3228 331.479,29.0603 334.213,28.8001 
 336.948,28.5424 339.682,28.287 342.417,28.0338 345.151,27.783 347.886,27.5343 350.62,27.2878 353.354,27.0435 356.089,26.8012 358.823,26.561 361.558,26.3228 
 364.292,26.0867 367.027,25.8524 369.761,25.6201 372.496,25.3898 375.23,25.1612 377.965,24.9345 380.699,24.7096 383.434,24.4865 386.168,24.2652 388.903,24.0455 
 391.637,23.8276 394.371,23.6113 397.106,23.3967 399.84,23.1837 402.575,22.9723 405.309,22.7625 408.044,22.5542 410.778,22.3474 413.513,22.1421 416.247,21.9384 
 418.982,21.736 421.716,21.5352 424.451,21.3357 427.185,21.1377 429.919,20.941 432.654,20.7457 435.388,20.5517 438.123,20.359 440.857,20.1677 443.592,19.9776 
 446.326,19.7888 449.061,19.6013 451.795,19.415 454.53,19.23 457.264,19.0461 459.999,18.8634 462.733,18.6819 465.467,18.5016 468.202,18.3224 470.936,18.1443 
 473.671,17.9674 476.405,17.7915 479.14,17.6168 481.874,17.4431 484.609,17.2705 487.343,17.0989 490.078,16.9284 492.812,16.7589 495.547,16.5904 498.281,16.4229 
 501.016,16.2564 503.75,16.0908 506.484,15.9263 509.219,15.7626 511.953,15.6 514.688,15.4382 517.422,15.2774 520.157,15.1175 522.891,14.9585 525.626,14.8004 
 528.36,14.6432 531.095,14.4868 533.829,14.3313 536.564,14.1767 539.298,14.0228 542.032,13.8699 544.767,13.7177 547.501,13.5664 550.236,13.4159 552.97,13.2661 
 555.705,13.1172 558.439,12.9691 561.174,12.8217 563.908,12.6751 566.643,12.5292 569.377,12.3841 572.112,12.2397 574.846,12.0961 577.58,11.9532 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip9802)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,273.034 38.8912,122.667 41.6256,116.21 44.3601,112.421 47.0946,109.743 49.829,107.66 52.5635,105.96 55.298,104.523 58.0324,103.278 60.7669,102.179 
 63.5014,101.195 66.2358,100.307 68.9703,99.4964 71.7047,98.749 74.4392,98.059 77.1737,97.4154 79.9081,96.8135 82.6426,96.2485 85.3771,95.7154 88.1115,95.2115 
 90.846,94.7328 93.5805,94.278 96.3149,93.8444 99.

## Finding a fixed point
This uses the previously defined `initial_w`, and finds the fixed point of the `T` mapping after binding the parameters

In [10]:
v_star_approx = compute_fixed_point(w -> T(w, params), #Creates a new function by fixing the parameters 
                                    initial_w,
                                    max_iter=500,
                                    verbose=2,
                                    print_skip=10,
                                    err_tol=1e-5)
plot(grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")

Compute iterate 10 with error 0.7061061807518572
Compute iterate 20 with error 0.4689419433411377
Compute iterate 30 with error 0.3117678508343076
Compute iterate 40 with error 0.2072734420817497
Compute iterate 50 with error 0.1378021488756076
Compute iterate 60 with error 0.0916153658786989
Compute iterate 70 with error 0.0609088852154116
Compute iterate 80 with error 0.040494214699162256
Compute iterate 90 with error 0.026921875506221937
Compute iterate 100 with error 0.017898541455497963
Compute iterate 110 with error 0.011899534496230046
Compute iterate 120 with error 0.00791119888650016
Compute iterate 130 with error 0.005259623213024867
Compute iterate 140 with error 0.00349676916260222
Compute iterate 150 with error 0.0023247662599743535
Compute iterate 160 with error 0.0015455804895161407
Compute iterate 170 with error 0.0010275523569696077
Compute iterate 180 with error 0.0006831503379487458
Compute iterate 190 with error 0.00045418063809776754
Compute iterate 200 with error 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -55 
 
 
 -50 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
<polyline clip-path="url(#clip0002)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,105.708 41.6256,93.4289 44.3601,86.2273 47.0946,81.1326 49.829,77.1768 52.5635,73.9424 55.298,71.2086 58.0324,68.8399 60.7669,66.7485 
 63.5014,64.8812 66.2358,63.1926 68.9703,61.6478 71.7047,60.2293 74.4392,58.9164 77.1737,57.6913 79.9081,56.5487 82.6426,55.4725 85.3771,54.4599 88.1115,53.4999 
 90.846,52.5917 93.5805,51.7258 96.3149,50.9019 99.0494,50.1127 101.784,49.3587 104.518,48.6347 107.253,47.9393 109.987,47.2708 112.722,46.6256 115.456,46.0038 
 118.191,45.4025 120.925,44.8208 123.66,44.2584 126.394,43.7127 129.128,43.1833 131.863,42.6696 134.597,42.1699 137.332,41.6839 140.066,41.2113 142.801,40.7505 
 145.535,40.3015 148.27,39.8641 151.004,39.4369 153.739,39.0197 156.473,38.6122 159.208,38.2138 161.942,37.8238 164.677,37.4426 167.411,37.0694 170.145,36.7038 
 172.88,36.3455 175.614,35.9945 178.349,35.6503 181.083,35.3126 183.818,34.9811 186.552,34.6559 189.287,34.3364 192.021,34.0224 194.756,33.714 197.49,33.4109 
 200.225,33.1129 202.959,32.8198 205.693,32.5315 208.428,32.2479 211.162,31.9687 213.897,31.6938 216.631,31.4231 219.366,31.1565 222.1,30.8938 224.835,30.635 
 227.569,30.3798 230.304,30.1282 233.038,29.8803 235.773,29.6357 238.507,29.3945 241.241,29.1565 243.976,28.9216 246.71,28.69 249.445,28.4613 252.179,28.2354 
 254.914,28.0125 257.648,27.7923 260.383,27.5747 263.117,27.3599 265.852,27.1475 268.586,26.9378 271.321,26.7304 274.055,26.5254 276.79,26.3228 279.524,26.1224 
 282.258,25.9244 284.993,25.7284 287.727,25.5347 290.462,25.3431 293.196,25.1535 295.931,24.9659 298.665,24.7802 301.4,24.5965 304.134,24.4146 306.869,24.2347 
 309.603,24.0564 312.338,23.8801 315.072,23.7053 317.806,23.5324 320.541,23.361 323.275,23.1913 326.01,23.0233 328.744,22.8568 331.479,22.6918 334.213,22.5284 
 336.948,22.3665 339.682,22.206 342.417,22.0471 345.151,21.8895 347.886,21.7333 350.62,21.5784 353.354,21.425 356.089,21.2728 358.823,21.1219 361.558,20.9723 
 364.292,20.8239 367.027,20.6768 369.761,20.5309 372.496,20.3862 375.23,20.2426 377.965,20.1002 380.699,19.9588 383.434,19.8187 386.168,19.6796 388.903,19.5417 
 391.637,19.4047 394.371,19.2689 397.106,19.1341 399.84,19.0003 402.575,18.8675 405.309,18.7357 408.044,18.6049 410.778,18.475 413.513,18.3461 416.247,18.218 
 418.982,18.091 421.716,17.9648 424.451,17.8395 427.185,17.7151 429.919,17.5916 432.654,17.4689 435.388,17.347 438.123,17.2261 440.857,17.1058 443.592,16.9865 
 446.326,16.8678 449.061,16.75 451.795,16.633 454.53,16.5167 457.264,16.4013 459.999,16.2865 462.733,16.1725 465.467,16.0592 468.202,15.9466 470.936,15.8348 
 473.671,15.7236 476.405,15.6131 479.14,15.5034 481.874,15.3943 484.609,15.2859 487.343,15.178 490.078,15.0709 492.812,14.9645 495.547,14.8586 498.281,14.7534 
 501.016,14.6488 503.75,14.5448 506.484,14.4414 509.219,14.3386 511.953,14.2364 514.688,14.1348 517.422,14.0338 520.157,13.9333 522.891,13.8334 525.626,13.7341 
 528.36,13.6353 531.095,13.5371 533.829,13.4394 536.564,13.3422 539.298,13.2456 542.032,13.1495 544.767,13.0539 547.501,12.9588 550.236,12.8643 552.97,12.7702 
 555.705,12.6767 558.439,12.5836 561.174,12.491 563.908,12.3989 566.643,12.3073 569.377,12.2161 572.112,12.1254 574.846,12.0352 577.58,11.9455 580.315,11.8562 
 
 "/>
<polyline clip-path="url(#clip0002)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,240.491 38.8912,105.643 41.6256,93.3591 44.3601,86.1724 47.0946,81.073 49.829,77.1175 52.5635,73.8856 55.298,71.153 58.0324,68.7859 60.7669,66.6979 
 63.5014,64.8302 66.2358,63.1406 68.9703,61.5981 71.7047,60.1792 74.4392,58.8654 77.1737,57.6423 79.9081,56.4982 82.6426,55.4235 85.3771,54.4102 88.1115,53.4517 
 90.846,52.5424

## Finding the Policy
This defines the optimal policy as the w-greedy argmin, where we use the "approximate value function" from before

In [13]:
#Note: This is almost entirely a copy of the previous T operator, except passing back the minimizer rather than the negative of the minimum.
#They could be integrated into a single function which returns both.

#Greedy operator
function T_σ(w_grid, params) #Greedy calculation given a w_grid
    grid, β, u, f, E = params.grid, params.β, params.u, params.f, params.E #unpack
    
    w = LinInterp(grid, w_grid) #linear interpolation of w_grid

    return [optimize(c -> - ( #Negates because this is a minimizer
                u(c) + β * E( ζ -> w(f(y - c) * exp(ζ)) ) #objective
                ), 1e-10, y).minimizer for y in grid] #for the whole grid
end

#A More verbose version of this funciton
function T_σ_verbose(w_grid, params)
    grid, β, u, f, E, c_min = params.grid, params.β, params.u, params.f, params.E, params.c_min #unpack
    
    w = LinInterp(grid, w_grid) #linear interpolation of w_grid
    c_grid = similar(w_grid) #Preallocating the result to be similar to w_grid
    
    for i in 1:length(w_grid)
        y = grid[i] #At each point on the grid
        c_max = y #i.e. the maximum value of c is y

        yprime(c, ζ) = f(y - c) * exp(ζ) #Law of motion for y
        objective(c) =  - (u(c) + β * E(ζ -> w(yprime(c, ζ)))) #objective negated to become a minimum
        
        result = optimize(objective, c_min, c_max);
        c_grid[i] = result.minimizer
    end
    return c_grid
end

σ = T_σ_verbose(v_star_approx, params)

plot(grid_y, σ, lw=2, alpha=0.6, label="approximate policy function")
plot!(grid_y, c_star.(grid_y), lw=2, alpha=0.6, label="true policy function")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
<polyline clip-path="url(#clip0602)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 32.4745,375.869 35.2275,374.024 37.9804,372.187 40.7334,370.384 43.4864,368.515 46.2393,366.731 48.9923,364.925 51.7453,363.049 54.4982,361.176 57.2512,359.34 
 60.0042,357.641 62.7571,355.784 65.5101,353.885 68.2631,352.141 71.016,350.236 73.769,348.437 76.522,346.614 79.2749,344.762 82.0279,342.974 84.7809,341.1 
 87.5338,339.311 90.2868,337.479 93.0398,335.615 95.7927,333.782 98.5457,331.902 101.299,330.162 104.052,328.211 106.805,326.465 109.558,324.609 112.311,322.75 
 115.064,320.98 117.816,319.22 120.569,317.32 123.322,315.553 126.075,313.558 128.828,311.863 131.581,310.113 134.334,308.068 137.087,306.366 139.84,304.588 
 142.593,302.754 145.346,300.793 148.099,298.995 150.852,297.206 153.605,295.412 156.358,293.601 159.111,291.744 161.864,289.779 164.617,288.003 167.37,286.24 
 170.123,284.396 172.876,282.546 175.629,280.772 178.382,279.008 181.135,277.173 183.888,275.332 186.641,273.559 189.394,271.776 192.147,269.894 194.9,268.037 
 197.653,266.237 200.405,264.429 203.158,262.536 205.911,260.632 208.664,258.815 211.417,257.009 214.17,255.12 216.923,253.226 219.676,251.41 222.429,249.646 
 225.182,247.693 227.935,246.101 230.688,244.178 233.441,242.442 236.194,240.585 238.947,238.628 241.7,236.664 244.453,235.097 247.206,233.057 249.959,231.479 
 252.712,229.513 255.465,227.592 258.218,225.845 260.971,223.977 263.724,222.32 266.477,220.413 269.23,218.466 271.983,216.802 274.736,214.812 277.489,213.232 
 280.242,211.237 282.995,209.233 285.747,207.654 288.5,205.563 291.253,203.966 294.006,201.965 296.759,200.398 299.512,198.376 302.265,196.584 305.018,194.805 
 307.771,192.86 310.524,191.213 313.277,189.197 316.03,187.609 318.783,185.459 321.536,184.002 324.289,181.811 327.042,180.352 329.795,178.148 332.548,176.688 
 335.301,174.501 338.054,173.045 340.807,170.832 343.56,169.375 346.313,167.159 349.066,165.699 351.819,163.509 354.572,162.052 357.325,159.839 360.078,158.382 
 362.831,156.162 365.584,154.701 368.336,152.495 371.089,151.038 373.842,148.793 376.595,147.336 379.348,145.152 382.101,143.619 384.854,141.731 387.607,139.91 
 390.36,138.111 393.113,136.163 395.866,134.558 398.619,132.405 401.372,130.945 404.125,128.673 406.878,127.179 409.631,125.07 412.384,123.45 415.137,121.637 
 417.89,119.686 420.643,117.9 423.396,115.949 426.149,114.453 428.902,112.342 431.655,110.722 434.408,108.736 437.161,106.943 439.914,105.066 442.667,103.188 
 445.42,101.691 448.173,99.5435 450.926,97.9239 453.678,96.066 456.431,94.1018 459.184,92.2308 461.937,90.3073 464.69,88.8091 467.443,86.6333 470.196,85.0044 
 472.949,83.3843 475.702,81.1905 478.455,79.6943 481.208,77.7994 483.961,75.8428 486.714,74.0731 489.467,72.2047 492.22,70.402 494.973,68.4575 497.726,66.7515 
 500.479,64.8063 503.232,62.9096 505.985,61.3918 508.738,59.1957 511.491,57.5603 514.244,55.9264 516.997,53.7181 519.75,52.2202 522.503,50.2883 525.256,48.32 
 528.009,46.8637 530.762,44.6217 533.515,42.9283 536.267,41.3417 539.02,39.0885 541.773,37.632 544.526,35.6428 547.279,33.6452 550.032,32.1886 552.785,29.9221 
 555.538,28.2828 558.291,26.627 561.044,24.376 563.797,22.8957 566.55,20.9045 569.303,19.3053 572.056,17.4335 574.809,15.1746 577.562,13.7114 580.315,11.8679 
 
 "/>
<polyline clip-path="url(#clip0602)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 32.4745,375.868 35.2275,374.039 37.9804,372.209 40.7334,370.38 43.4864,368.55 46.2393,366.721 48.9923,364.891 51.7453,363.062 54.4982,361.233 57.2512,359.403 
 60.0042,357.574 62.7571,355.744 65.5101,353.915 68.2631,352.085 71.016,350.256 73.769,348.427 76.522,346.597 79.2749,344.768 82.0279,342.938 84.7809,341.109 
 87.5338,339.279 90.2868,337.45 93.0398,335.621 95.7927,333.791 98

## Finally, we simulate the evolution of the state 
This will work for an arbitrary production function and distribution in the parameters

In [104]:
function simulate_og(σ, params, y0 = 0.1, ts_length=100)
    f, dist = params.f, params.dist #For arbitrary f and distributions
    
    ξ = rand(dist, ts_length - 1) #Draws random numbers from the distribution
    y = zeros(ts_length)   
    y[1] = y0
    
    for t in 1:(ts_length-1)
        y[t+1] = f(y[t] - σ(y[t])) * exp(ξ[t])
    end
    return y
end

σ_func = LinInterp(grid_y, σ)
y = simulate_og(σ_func, params)

plot(y, lw=2, alpha=0.6, label="\\beta = $β" )

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
<polyline clip-path="url(#clip5102)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 32.4745,375.869 38.0082,232.828 43.542,116.476 49.0757,58.7403 54.6095,31.2988 60.1432,32.1877 65.6769,16.402 71.2107,19.2558 76.7444,22.226 82.2782,18.0222 
 87.8119,12.3542 93.3457,19.2771 98.8794,25.6639 104.413,17.6805 109.947,16.6071 115.481,22.7399 121.014,17.245 126.548,18.2583 132.082,22.2625 137.616,14.0305 
 143.149,15.456 148.683,19.1324 154.217,24.6127 159.751,19.2872 165.284,29.3193 170.818,19.0595 176.352,15.4113 181.886,17.2167 187.419,15.6415 192.953,14.2923 
 198.487,23.323 204.02,22.8974 209.554,14.2289 215.088,16.0803 220.622,24.2958 226.155,16.4829 231.689,18.8726 237.223,23.6364 242.757,26.6304 248.29,18.349 
 253.824,20.032 259.358,27.2112 264.892,13.1699 270.425,12.8502 275.959,17.0014 281.493,16.4042 287.027,25.8164 292.56,20.8318 298.094,14.7371 303.628,15.8464 
 309.162,23.5227 314.695,25.2502 320.229,14.7591 325.763,19.8777 331.297,19.2455 336.83,15.9145 342.364,17.7888 347.898,18.0844 353.432,13.0208 358.965,18.3943 
 364.499,19.6352 370.033,19.817 375.567,23.3677 381.1,17.4548 386.634,23.3337 392.168,21.6198 397.701,16.8611 403.235,18.2234 408.769,14.2007 414.303,13.957 
 419.836,18.2165 425.37,23.495 430.904,21.1948 436.438,11.811 441.971,16.6959 447.505,23.2757 453.039,26.6009 458.573,18.5571 464.106,23.2598 469.64,16.4373 
 475.174,22.844 480.708,16.9857 486.241,22.5695 491.775,19.7033 497.309,23.8122 502.843,26.1304 508.376,16.072 513.91,22.6749 519.444,19.2126 524.978,21.387 
 530.511,18.1465 536.045,20.5867 541.579,24.3015 547.113,23.6833 552.646,22.7266 558.18,18.615 563.714,16.6992 569.247,20.801 574.781,20.8167 580.315,13.9873 
 
 "/>
 
 
 
 
 β 
 
 
 = 0.96